In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
import os
from torchvision.models import efficientnet_b0
import timm
import matplotlib.pyplot as plt
import numpy as np
import cv2
import time
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns
import utility_modele as md
from sklearn.model_selection import train_test_split
import json

import os
import time
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
import timm
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [13]:
csv_path = "/kaggle/input/rayon-x-label/DB_patients_norm_bbox.csv"
df = pd.read_csv(csv_path)

In [15]:
def filtrer_maladie_unique(df, maladie_cible, autres_maladies):
    """
    Retourne les lignes du DataFrame où :
    - maladie_cible == 1
    - toutes les autres maladies == 0
    """
    condition_cible = df[maladie_cible] == 1
    autres = [m for m in autres_maladies if m != maladie_cible]
    condition_autres = df[autres].sum(axis=1) == 0

    return df[condition_cible & condition_autres]

maladies = [
    'Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration', 'Mass',
    'Nodule', 'Pneumonia', 'Pneumothorax', 'Consolidation', 'Edema',
    'Emphysema', 'Fibrosis', 'Pleural_Thickening', 'Hernia', 'No Finding'
]

df_atelectasis_seule = filtrer_maladie_unique(df, "Atelectasis", maladies)
df_no_finding_seule = filtrer_maladie_unique(df, "No Finding", maladies)
df_no_finding_seule = df_no_finding_seule.sample(n=5000, random_state=42)
df_atelectasis_final = pd.concat([df_atelectasis_seule, df_no_finding_seule], ignore_index=True)

In [16]:
class CLAHETransform:
    def __init__(self, clip_limit=2.0, tile_grid_size=(8, 8)):
        self.clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)

    def __call__(self, img):
        # Convert PIL to grayscale numpy
        img_np = np.array(img.convert('L'))  # convert to grayscale
        img_clahe = self.clahe.apply(img_np)

        # Reconvert to PIL with 3 channels (fake RGB)
        img_rgb = Image.fromarray(cv2.cvtColor(img_clahe, cv2.COLOR_GRAY2RGB))
        return img_rgb

def detect_lung_roi(image_pil):
    img_gray = np.array(image_pil.convert('L'))  # PIL → grayscale numpy
    img_blur = cv2.GaussianBlur(img_gray, (5, 5), 0)
    _, thresh = cv2.threshold(img_blur, 30, 255, cv2.THRESH_BINARY)

    # Trouver les contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return image_pil  # fallback

    # Récupérer le plus grand contour
    largest = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest)

    # Crop l'image selon le ROI détecté
    img_cropped = img_gray[y:y+h, x:x+w]
    img_rgb = cv2.cvtColor(img_cropped, cv2.COLOR_GRAY2RGB)
    return Image.fromarray(img_rgb)

class LungROICrop:
    def __call__(self, img):
        return detect_lung_roi(img)

class CenterCropZoom:
    def __init__(self, zoom_factor=1.2):
        self.zoom_factor = zoom_factor

    def __call__(self, img):
        w, h = img.size
        new_w = int(w / self.zoom_factor)
        new_h = int(h / self.zoom_factor)
        left = (w - new_w) // 2
        top = (h - new_h) // 2
        right = left + new_w
        bottom = top + new_h
        img_cropped = img.crop((left, top, right, bottom))
        return img_cropped

In [ ]:
df_atelectasis_final["Finding Labels"].value_counts()

In [17]:
# 1. Dataset PyTorch multi-label
class ChestXrayDataset(Dataset):
    def __init__(self, dataframe, label_cols, transform=None):
        self.df = dataframe
        self.label_cols = label_cols
        if transform is None:
            self.transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
            ])
        else:
            self.transform = transform    

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(row['Image Path']).convert('RGB')
        if self.transform:
            image = self.transform(image)
        labels = torch.tensor(row[self.label_cols].astype(np.float32).values, dtype=torch.float32)
        return image, labels

    def __afficher_image__(self, idx):
        """
        Affiche l'image (dénormalisée) et ses labels associés.
        """
        image, label = self[idx]

        # # Dénormalisation
        mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
        std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
        image = image * std + mean  # Inverser la normalisation

        # Passage à la forme (H, W, C) pour matplotlib
        image = image.permute(1, 2, 0).clamp(0, 1)

        # Affichage
        plt.imshow(image)
        plt.title(f"Label: {label}")
        plt.axis('off')
        plt.show()

        # Affichage des maladies détectées
        nb_maladie = 0
        for i in range(len(label)):
            if label[i] == 1:
                print("Maladies présentes :")
                print(f" - {self.label_cols[i]}")
                nb_maladie += 1
        if(nb_maladie == 0):
            print("Aucune maladie")

# 3. Transforms
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.RandomHorizontalFlip(p=0.5),
#     transforms.RandomRotation(10),
#     transforms.RandomAffine(degrees=10, translate=(0.05, 0.05), scale=(0.95, 1.05)),
#     transforms.ColorJitter(brightness=0.2, contrast=0.2),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
# ])


transform = transforms.Compose([
    LungROICrop(),
    CLAHETransform(),
    transforms.Resize((256, 256)),  # Agrandit
    CenterCropZoom(zoom_factor=1.2),  # Zoom centré
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.2),
    transforms.RandomRotation(5),
    transforms.RandomAffine(degrees=10, translate=(0.05, 0.05), scale=(0.95, 1.05)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

# 4. Split des données
# 2. Préparation des données
label_columns = ['Atelectasis']

train_df, val_df = train_test_split(df_atelectasis_final, test_size=0.2, random_state=42)


train_dataset = ChestXrayDataset(train_df, label_columns, transform)
val_dataset = ChestXrayDataset(val_df, label_columns, transform)


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


# 5. Modèle ViT
model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=1)
model = model.cuda()


ratio_neg, ratio_pos = df_atelectasis_final["Finding Labels"].value_counts()
pos_weight = torch.tensor([ratio_neg/ratio_pos]).cuda()
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

best_f1 = 0
train_losses = []
val_losses = []

# 7. Validation

def evaluate(model, loader):
    model.eval()
    all_labels, all_outputs = [], []
    total_loss = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.cuda(), labels.cuda()
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            outputs = torch.sigmoid(outputs).cpu().numpy()
            all_outputs.extend(outputs)
            all_labels.extend(labels.cpu().numpy())
    val_loss = total_loss / len(loader)
    auc = roc_auc_score(all_labels, all_outputs)
    preds = [1 if o >= 0.5 else 0 for o in all_outputs]
    f1 = f1_score(all_labels, preds)
    precision = precision_score(all_labels, preds)
    recall = recall_score(all_labels, preds)
    accuracy = accuracy_score(all_labels, preds)
    return val_loss, auc, f1, precision, recall, accuracy

In [18]:
# 8. Boucle d'entraînement
patience = 5
best_f1 = 0
epochs_no_improve = 0
max_epochs = 50

train_losses = []
val_losses = []

for epoch in range(max_epochs):
    model.train()
    epoch_loss = 0
    start_time = time.time()
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        images, labels = images.cuda(), labels.cuda()
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    epoch_loss /= len(train_loader)
    train_losses.append(epoch_loss)

    val_loss, auc, f1, precision, recall, accuracy = evaluate(model, val_loader)
    val_losses.append(val_loss)
    elapsed = time.time() - start_time

    print(f"Epoch {epoch+1} - Time: {elapsed:.2f}s - Loss: {epoch_loss:.4f} - Val Loss: {val_loss:.4f} - AUC: {auc:.4f} - F1: {f1:.4f} - Precision: {precision:.4f} - Recall: {recall:.4f} - Accuracy: {accuracy:.4f}")

    if f1 > best_f1:
        best_f1 = f1
        epochs_no_improve = 0
        torch.save(model.state_dict(), "best_model.pth")
        with open("best_metrics.txt", "w") as f:
            f.write(f"Epoch: {epoch+1}\nLoss: {epoch_loss:.4f}\nVal Loss: {val_loss:.4f}\nAUC: {auc:.4f}\nF1: {f1:.4f}\nPrecision: {precision:.4f}\nRecall: {recall:.4f}\nAccuracy: {accuracy:.4f}\n")
    else:
        epochs_no_improve += 1

    if epochs_no_improve >= patience:
        print("Early stopping activé après", patience, "époques sans amélioration.")
        break

Epoch 1:   3%|▎         | 7/231 [00:16<09:00,  2.41s/it]


KeyboardInterrupt: 

In [ ]:
with open("/kaggle/working/best_metrics.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        print(line.strip())


In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Courbe des pertes durant l\'entraînement')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# 10. Attention Rollout pour ViT
class VitAttentionRollout:
    def __init__(self, model, head_fusion='mean'):
        self.model = model
        self.head_fusion = head_fusion
        self.attentions = []
        self._register_hooks()

    def _register_hooks(self):
        for blk in self.model.blocks:
            blk.attn.register_forward_hook(self._get_attention)


    def _get_attention(self, module, input, output):
        self.attentions.append(output.detach().cpu())

    def __call__(self, input_tensor):
        self.attentions = []
        _ = self.model(input_tensor)
        result = torch.eye(self.attentions[0].size(-1))
        for attention in self.attentions:
            if self.head_fusion == 'mean':
                attention_heads_fused = attention.mean(dim=1)
            else:
                raise ValueError("head_fusion must be 'mean'")
            attention_heads_fused = attention_heads_fused + torch.eye(attention_heads_fused.size(-1))
            attention_heads_fused = attention_heads_fused / attention_heads_fused.sum(dim=-1, keepdim=True)
            result = torch.matmul(attention_heads_fused, result)
        mask = result[0, 1:]
        n_tokens = mask.shape[0]
        side = int(n_tokens ** 0.5)
        return mask[:side*side].reshape(side, side).numpy()

In [ ]:
# def __getitem__(self, idx):
#     row = self.df.iloc[idx]
#     image = Image.open(row['Image Path']).convert('RGB')
#     if self.transform:
#         image = self.transform(image)
#     labels = torch.tensor(row[self.label_cols].astype(np.float32).values, dtype=torch.float32)
#     return image, labels

img =  Image.open("/kaggle/input/data/images_004/images/00006904_007.png").convert('RGB')
a = transform(img)

In [ ]:
# 11. Application de l'explicabilité
model.eval()
rollout = VitAttentionRollout(model)

sample_image = a  # 'a' est une image transformée déjà prête (Tensor)
sample_image_input = sample_image.unsqueeze(0).cuda()

# Prédiction
with torch.no_grad():
    output = model(sample_image_input)
    prob = torch.sigmoid(output).item()

# Attention rollout
attention_mask = rollout(sample_image_input)

# Visualisation
plt.imshow(sample_image.permute(1, 2, 0).cpu().numpy().clip(0, 1))
plt.imshow(attention_mask, cmap='jet', alpha=0.5, extent=(0, 224, 224, 0))
plt.title(f"Attention Rollout - ViT\nProbabilité d'Atelectasis : {prob:.2f}")
plt.axis('off')
plt.show()


In [ ]:
def predict_image(image_path):
    model.eval()
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).cuda()
    with torch.no_grad():
        output = model(image)
        prob = torch.sigmoid(output).item()
    print(f"Probabilité de présence d'Atelectasis: {prob:.4f}")
    if prob >= 0.5:
        print("=> Maladie détectée (Atelectasis)")
    else:
        print("=> Aucune maladie détectée")

predict_image("/kaggle/input/data/images_004/images/00006904_007.png")

In [ ]:
df_atelectasis_final[df_atelectasis_final["Atelectasis"] == 1]

In [ ]:
model_vit = timm.create_model('vit_base_patch16_224', pretrained=False, num_classes=1)

# 2. Charger les poids
state_dict = torch.load("/kaggle/working/best_model.pth", map_location="cpu")
model_vit.load_state_dict(state_dict)

# 3. Sauvegarder en .pt (modèle complet)
torch.save(model_vit, "best_model_vit.pt")

In [ ]:
from IPython.display import FileLink
FileLink("/kaggle/working/best_model_vit.pt")

In [ ]:
label_columns = ["Atelectasis"]
model1 = efficientnet_b0(pretrained=False)
model1.classifier[1] = nn.Linear(model1.classifier[1].in_features, len(label_columns))

# 2. Charger les poids sur CPU
state_dict = torch.load(
    "/kaggle/working/efficientnetb0_appa_atelectasis_v3.pt",
    map_location=torch.device("cpu")
)
model1.load_state_dict(state_dict)

# 3. Mettre sur le bon device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model1 = model1.to(device)
model1.eval()

md.afficher_metrics_json("/kaggle/working/efficientnetb0_appa_atelectasis_v3.json")

In [ ]:
import torch
import torch.nn as nn
from timm import create_model

# 1. Définir les labels
label_columns = ["Atelectasis"]

# 2. Créer le modèle ViT (ex: vit_base_patch16_224)
model_vit = create_model(
    "vit_base_patch16_224",     # ⚠️ à adapter si tu as utilisé une autre version
    pretrained=False,
    num_classes=len(label_columns)
)

# 3. Charger les poids (sur CPU)
state_dict = torch.load(
    "/kaggle/input/vit-atelectasis/pytorch/default/1/vit_at.pth", 
    map_location=torch.device("cpu")
)
model_vit.load_state_dict(state_dict)

# 4. Mettre le modèle sur le bon device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_vit = model_vit.to(device)
model_vit.eval()

In [ ]:
# 10. Attention Rollout pour ViT
class VitAttentionRollout:
    def __init__(self, model, head_fusion='mean'):
        self.model = model
        self.head_fusion = head_fusion
        self.attentions = []
        self._register_hooks()

    def _register_hooks(self):
        for blk in self.model.blocks:
            blk.attn.register_forward_hook(self._get_attention)


    def _get_attention(self, module, input, output):
        self.attentions.append(output.detach().cpu())

    def __call__(self, input_tensor):
        self.attentions = []
        _ = self.model(input_tensor)
        result = torch.eye(self.attentions[0].size(-1))
        for attention in self.attentions:
            if self.head_fusion == 'mean':
                attention_heads_fused = attention.mean(dim=1)
            else:
                raise ValueError("head_fusion must be 'mean'")
            attention_heads_fused = attention_heads_fused + torch.eye(attention_heads_fused.size(-1))
            attention_heads_fused = attention_heads_fused / attention_heads_fused.sum(dim=-1, keepdim=True)
            result = torch.matmul(attention_heads_fused, result)
        mask = result[0, 1:]
        n_tokens = mask.shape[0]
        side = int(n_tokens ** 0.5)
        return mask[:side*side].reshape(side, side).numpy()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_vit = model_vit.to(device)  # ⚠️ Important

# Image -> Tensor
img = Image.open("/kaggle/input/data/images_004/images/00006904_007.png").convert('RGB')
a = transform(img)
sample_image_input = a.unsqueeze(0).to(device)  # 👈 sur le même device que le modèle

# Prédiction
with torch.no_grad():
    output = model_vit(sample_image_input)
    prob = torch.sigmoid(output).item()

# Attention rollout
rollout = VitAttentionRollout(model_vit)  # s'assurer qu'il est aussi sur le bon device
attention_mask = rollout(sample_image_input)

# Visualisation
plt.imshow(a.permute(1, 2, 0).cpu().numpy().clip(0, 1))
plt.imshow(attention_mask, cmap='jet', alpha=0.5, extent=(0, 224, 224, 0))
plt.title(f"Attention Rollout - ViT\nProbabilité d'Atelectasis : {prob:.2f}")
plt.axis('off')
plt.show()


In [ ]:
import torch
import matplotlib.pyplot as plt
from PIL import Image
from timm import create_model
from scipy.ndimage import zoom

# 1. Charger l'image et la transformer
img = Image.open("/kaggle/input/data/images_004/images/00006904_007.png").convert('RGB')
a = transform(img)  # transform = Resize, ToTensor, Normalize, etc.
sample_image_input = a.unsqueeze(0).to(device)

# 2. Prédiction
model_vit.eval()
with torch.no_grad():
    output = model_vit(sample_image_input)
    prob = torch.sigmoid(output).item()

# 3. Attention Rollout
rollout = VitAttentionRollout(model_vit)
attention_mask = rollout(sample_image_input)  # shape: [14, 14] ou similaire

# 4. Redimensionner la heatmap vers la taille de l’image
# sample_image a la forme [C, H, W]
h, w = a.shape[1], a.shape[2]
zoom_factors = (h / attention_mask.shape[0], w / attention_mask.shape[1])
attention_mask_resized = zoom(attention_mask, zoom_factors)

# 5. Affichage image + heatmap lissée
plt.imshow(a.permute(1, 2, 0).cpu().numpy().clip(0, 1))  # Image RGB
plt.imshow(attention_mask_resized, cmap='jet', alpha=0.5, interpolation='bilinear')  # Heatmap
plt.title(f"Attention Rollout - ViT\nProbabilité d'Atelectasis : {prob:.2f}")
plt.axis('off')
plt.show()

# Multi label

In [19]:
def supprimer_lignes_pathologies(df):
    colonnes_exclues = [
        'Hernia', 'Pneumonia', 'Fibrosis',
        'Edema', 'Emphysema', 'Cardiomegaly',
        'Pleural_Thickening'
    ]
    
    # Supprimer les lignes qui ont au moins un 1 dans les colonnes concernées
    df_filtré = df[~(df[colonnes_exclues].sum(axis=1) > 0)].reset_index(drop=True)
    
    return df_filtré
df = supprimer_lignes_pathologies(df)
df = df[df["View Position"] == "PA"]
df.shape

no_finding_df = df[df["No Finding"] == 1]

# 2. Sélectionner aléatoirement 8000 lignes parmi elles
no_finding_sample = no_finding_df.sample(n=8000, random_state=42)

# 3. Séparer le reste du DataFrame (tout sauf les lignes "No Finding" == 1)
other_df = df[df["No Finding"] != 1]

# 4. Recombiner le tout
df = pd.concat([other_df, no_finding_sample], ignore_index=True)

df["Finding Labels"].value_counts()

Finding Labels
No Finding                                                     8000
Infiltration                                                   5270
Atelectasis                                                    2210
Effusion                                                       2086
Nodule                                                         1924
                                                               ... 
Nodule|Mass                                                       1
Atelectasis|Mass|Nodule|Pneumothorax                              1
Atelectasis|Consolidation|Effusion|Mass|Nodule|Pneumothorax       1
Mass|Nodule|Atelectasis                                           1
Effusion|Nodule|Pneumothorax|Mass                                 1
Name: count, Length: 118, dtype: int64

In [21]:
# 1. Dataset PyTorch multi-label
class ChestXrayDataset(Dataset):
    def __init__(self, dataframe, label_cols, transform=None):
        self.df = dataframe
        self.label_cols = label_cols
        if transform is None:
            self.transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
            ])
        else:
            self.transform = transform    

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(row['Image Path']).convert('RGB')
        if self.transform:
            image = self.transform(image)
        labels = torch.tensor(row[self.label_cols].astype(np.float32).values, dtype=torch.float32)
        return image, labels

    def __afficher_image__(self, idx):
        """
        Affiche l'image (dénormalisée) et ses labels associés.
        """
        image, label = self[idx]

        # # Dénormalisation
        mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
        std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
        image = image * std + mean  # Inverser la normalisation

        # Passage à la forme (H, W, C) pour matplotlib
        image = image.permute(1, 2, 0).clamp(0, 1)

        # Affichage
        plt.imshow(image)
        plt.title(f"Label: {label}")
        plt.axis('off')
        plt.show()

        # Affichage des maladies détectées
        nb_maladie = 0
        for i in range(len(label)):
            if label[i] == 1:
                print("Maladies présentes :")
                print(f" - {self.label_cols[i]}")
                nb_maladie += 1
        if(nb_maladie == 0):
            print("Aucune maladie")

# 3. Transforms
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.RandomHorizontalFlip(p=0.5),
#     transforms.RandomRotation(10),
#     transforms.RandomAffine(degrees=10, translate=(0.05, 0.05), scale=(0.95, 1.05)),
#     transforms.ColorJitter(brightness=0.2, contrast=0.2),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
# ])


transform = transforms.Compose([
    LungROICrop(),
    CLAHETransform(),
    transforms.Resize((256, 256)),  # Agrandit
    CenterCropZoom(zoom_factor=1.2),  # Zoom centré
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.2),
    transforms.RandomRotation(5),
    transforms.RandomAffine(degrees=10, translate=(0.05, 0.05), scale=(0.95, 1.05)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

# 4. Split des données
# 2. Préparation des données
label_columns = ['Atelectasis', 'Infiltration', 'Effusion',  'Mass',
    'Nodule', 'Pneumothorax', 'Consolidation']

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)


train_dataset = ChestXrayDataset(train_df, label_columns, transform)
val_dataset = ChestXrayDataset(val_df, label_columns, transform)


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


# 5. Modèle ViT
model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=len(label_columns))
model = model.cuda()


counts_pos = df[label_columns].sum()
counts_neg = df.shape[0] - counts_pos

pos_weight_tensor = torch.tensor((counts_neg / counts_pos).values, dtype=torch.float32).cuda()
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight_tensor)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

best_f1 = 0
train_losses = []
val_losses = []

# 7. Validation

def evaluate(model, loader):
    model.eval()
    all_labels, all_outputs = [], []
    total_loss = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.cuda(), labels.cuda()
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            outputs = torch.sigmoid(outputs).cpu().numpy()
            all_outputs.extend(outputs)
            all_labels.extend(labels.cpu().numpy())
    val_loss = total_loss / len(loader)
    auc = roc_auc_score(all_labels, all_outputs)
    preds = (np.array(all_outputs) >= 0.5).astype(int)

    f1 = f1_score(all_labels, preds)
    precision = precision_score(all_labels, preds)
    recall = recall_score(all_labels, preds)
    accuracy = accuracy_score(all_labels, preds)
    return val_loss, auc, f1, precision, recall, accuracy

In [ ]:
# 8. Boucle d'entraînement
patience = 5
best_f1 = 0
epochs_no_improve = 0
max_epochs = 50

train_losses = []
val_losses = []

for epoch in range(max_epochs):
    model.train()
    epoch_loss = 0
    start_time = time.time()
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        images, labels = images.cuda(), labels.cuda()
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    epoch_loss /= len(train_loader)
    train_losses.append(epoch_loss)

    val_loss, auc, f1, precision, recall, accuracy = evaluate(model, val_loader)
    val_losses.append(val_loss)
    elapsed = time.time() - start_time

    print(f"Epoch {epoch+1} - Time: {elapsed:.2f}s - Loss: {epoch_loss:.4f} - Val Loss: {val_loss:.4f} - AUC: {auc:.4f} - F1: {f1:.4f} - Precision: {precision:.4f} - Recall: {recall:.4f} - Accuracy: {accuracy:.4f}")

    if f1 > best_f1:
        best_f1 = f1
        epochs_no_improve = 0
        torch.save(model.state_dict(), "best_model.pth")
        with open("best_metrics.txt", "w") as f:
            f.write(f"Epoch: {epoch+1}\nLoss: {epoch_loss:.4f}\nVal Loss: {val_loss:.4f}\nAUC: {auc:.4f}\nF1: {f1:.4f}\nPrecision: {precision:.4f}\nRecall: {recall:.4f}\nAccuracy: {accuracy:.4f}\n")
    else:
        epochs_no_improve += 1

    if epochs_no_improve >= patience:
        print("Early stopping activé après", patience, "époques sans amélioration.")
        break

Epoch 1: 100%|██████████| 715/715 [22:30<00:00,  1.89s/it]
